# Installing Packages and Dependencies

In [ ]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://download.pytorch.org/whl/cu121
  Using cached https://download.pytorch.org/whl/cu121/torchvision-0.17.0%2Bcu121-cp310-cp310-win_amd64.whl (5.7 MB)
  Using cached https://download.pytorch.org/whl/cu121/torchaudio-2.2.0%2Bcu121-cp310-cp310-win_amd64.whl (4.1 MB)
     ---------------------------------------- 2.5/2.5 GB 516.7 kB/s eta 0:00:00


In [2]:
!pip install transformers requests beautifulsoup4 pandas numpy

Defaulting to user installation because normal site-packages is not writeable


In [11]:
from transformers import AutoTokenizer,AutoModelForSequenceClassification
import torch
import requests
from bs4 import BeautifulSoup
import re

## Instantinate Model

In [12]:
tokenizer=AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')
model=AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

## Encode and Calculate Sentiment

In [13]:
tokens=tokenizer.encode('Meh, it was OK',return_tensors='pt')

In [14]:
result=model(tokens)

In [15]:
result.logits

tensor([[-0.7896,  1.6718,  2.7634, -0.2691, -2.8158]],
       grad_fn=<AddmmBackward0>)

In [16]:
int(torch.argmax(result.logits))+1

3

## Collect Reviews

In [62]:
r = requests.get('https://b210dba515cbbf9e26.gradio.live/')# link obtained in the ChatBot Notebook
soup = BeautifulSoup(r.text, 'html.parser')
regex = re.compile('.*message.*')
results = soup.find_all('p', {'class':regex})
reviews = [result.text for result in results]

## Load Reviews into DataFrame and Score

In [63]:
import numpy as np
import pandas as pd

In [64]:
df = pd.DataFrame(np.array(reviews), columns=['review'])

In [65]:
print(df.shape)

(0, 1)


In [66]:
df['review'].iloc[0]

IndexError: single positional indexer is out-of-bounds

def sentiment_score(review):
    tokens = tokenizer.encode(review, return_tensors='pt')
    result = model(tokens)
    return int(torch.argmax(result.logits))+1

In [67]:
sentiment_score(df['review'].iloc[1])

IndexError: single positional indexer is out-of-bounds

In [68]:
df['sentiment'] = df['review'].apply(lambda x: sentiment_score(x[:512]))

In [69]:
df

,review,sentiment
